In [ ]:
import gflex
import pandas as pd
import numpy as np
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

from scripts import morphoGrid as morph
from scipy.spatial import cKDTree

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

# Define parameters

This should mimick your input file conditions.

In [ ]:
dx = 50000 # resolution
step = 800 # Here we only analyse 1 step i.e. output number from badlands
badlands_run = 'sunda_spratt/h5'


# Parameters for gflex
poisson = 0.25
youngMod = 65e9
mantleDensity = 3300.0
sedimentDensity = 2500.0
elasticT = 35000.
boundary_W = '0Displacement0Slope'
boundary_E = '0Displacement0Slope'
boundary_S = '0Displacement0Slope'
boundary_N = '0Displacement0Slope'

We read the output and create a regular grid to run the flexure...

In [ ]:
badland_topo = morph.morphoGrid(folder=badlands_run, bbox = None, dx=dx)
badland_topo.loadHDF5(timestep=step)

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = plt.gca()
im = ax.imshow(badland_topo.z-badland_topo.sl, cmap ='RdYlBu_r', vmin = -3000, vmax = 3000, 
                    interpolation ='nearest', origin ='lower') 

ax.contour(badland_topo.z-badland_topo.sl, levels=[0], colors='k', linewidths=1)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

plt.colorbar(im, cax=cax)
plt.show()

# Compute the flexure

This is just a one step example here where I show how the flexure is calculated at each time step. In your case the obtained flexure has to be substracted to the previous time step... I do that in the second notebook.

In [ ]:
flex = gflex.F2D()
# Set-up the grid variable
flex.dx = dx
flex.dy = dx

# Solution method finite difference
flex.Method = "FD"
flex.Quiet = True

# van Wees and Cloetingh (1994)
flex.PlateSolutionType = "vWC1994"
flex.Solver = "direct"

# Acceleration due to gravity
flex.g = 9.8
# Poisson's Ratio
flex.nu = poisson
# Infill Material Density
flex.rho_fill = 0.0
# Young's Modulus
flex.E = youngMod
# Mantle Density
flex.rho_m = mantleDensity
# Sediment Density
rho_s = sedimentDensity
# Sea Water Density
rho_w = 1029.0

Te = elasticT * np.ones(badland_topo.z.shape)

flex.Te = Te

# Surface load stresses
flex.qs = np.zeros(badland_topo.z.shape, dtype=float)

# Boundary conditions
flex.BC_W = boundary_W
flex.BC_E = boundary_E
flex.BC_S = boundary_S
flex.BC_N = boundary_N

# State of the previous flexural grid used for updating current
# flexural displacements.
previous_flex = np.zeros(badland_topo.z.shape, dtype=float)
waterload = np.zeros(badland_topo.z.shape)

marine = np.where(badland_topo.z < badland_topo.sl)
waterload[marine] = badland_topo.sl - badland_topo.z[marine]

# Compute surface loads
flex.qs = (rho_w * flex.g * waterload)

In [ ]:
flex.initialize()
flex.run()
flex.finalize()

Let's plot the flexural response due to water loading for this specific step... this is considering that there was no water prior to this step (as we don't account for the previous step) but this notebook is just here to show the steps done in the loop in the next notebook.

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = plt.gca()
im = ax.imshow(flex.w, cmap ='RdYlBu_r', vmin = -3000, vmax = 3000, 
                    interpolation ='nearest', origin ='lower') 

ax.contour(badland_topo.z-badland_topo.sl, levels=[0], colors='k', linewidths=1)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

plt.colorbar(im, cax=cax)
plt.show()